In [ ]:
#Defining CNN Model

from keras.models import Sequential    #For initialising the model
from keras.layers import Conv2D        #For adding convolutional layer
from keras.layers import MaxPooling2D  #For adding max pooling layer
from keras.layers import Flatten       #For flattening max pooled layer values into a single vector
from keras.layers import Dense         #For adding layers to NN

h_layers = 1   #no. of hidden layers
features = 32  #no. of feature maps 
neurons = 128  #no. of neurons in each hidden layer

model = Sequential() #initialise the model

model.add( Conv2D( features, (3, 3),input_shape = (64, 64, 1), activation = 'relu' )) #Dims of feature map = 3*3, accepting 64*64 pixels grayscale images
model.add( MaxPooling2D( pool_size = (2, 2) )) #add max pooling layer, with dims of each pool = 2*2

model.add( Flatten() ) #add flattening layer 
    
for i in range( h_layers ):  #add all hidden layers
    model.add( Dense( units = neurons, activation = 'relu' ))
        
model.add( Dense( units = 1, activation = 'sigmoid' ))  #add an output layer

model.compile( optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )  #define optimizer and loss functions as well as required metrics


In [ ]:
#Converting all images into grayscale JPEG format

from PIL import Image  #For manipulating images
import glob            #For accessing all image files 

#Converting all images in training as well as testing sets to grayscale JPEG format
for fil in glob.glob(r'dataset\train_set\yes\*.*'):
    img = Image.open(fil)
    img = img.convert('L') 
    img.save(fil, format = 'JPEG')

for fil in glob.glob(r'dataset\train_set\no\*.*'):
    img = Image.open(fil)
    img = img.convert('L') 
    img.save(fil, format = 'JPEG')

for fil in glob.glob(r'dataset\test_set\yes\*.*'):
    img = Image.open(fil)
    img = img.convert('L') 
    img.save(fil, format = 'JPEG')

for fil in glob.glob(r'dataset\test_set\no\*.*'):
    img = Image.open(fil)
    img = img.convert('L') 
    img.save(fil, format = 'JPEG')


In [ ]:
#Image Augmentation and converting images so that they can be fed into the CNN

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1. / 255)  #Rescaling all pixel values from 0-255 range to 0-1 range

train = datagen.flow_from_directory('dataset/train_set',        #creating training data               
                                     target_size=(64, 64),
                                     batch_size=32,
                                     color_mode='grayscale',
                                     class_mode='binary')

test = datagen.flow_from_directory('dataset/test_set',          #creating testing data
                                    target_size=(64, 64),
                                    batch_size=32,
                                    color_mode='grayscale',
                                    class_mode='binary')


In [ ]:
#Fitting model to training and testing data and saving the model

history = model.fit_generator(train,
                    steps_per_epoch=181,
                    epochs=50,
                    validation_data=test,
                    validation_steps=78)

import h5py
model.save('cnn_model.h5')

In [ ]:
#Plotting Training and Testing accuracies

import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='middle right')
plt.show()